**<font  size=6>
使用Excel表格讀取並合併
</font>**

In [1]:
import pandas as pd

In [15]:
def make_melt_table(table_path,feature_names):
    #讀取數據
    df2000 = pd.read_excel(table_path,index_col=0).reset_index(drop =True).iloc[:,3:].reset_index(drop =True)
    df2000['日期'] = df2000['股票代號'].str[0:8]
    stock_name_table = pd.DataFrame({'股票代號':df2000.columns[1:-2],'股票名稱':df2000.iloc[0,1:-2]}).reset_index(drop = True)
    df2000 = df2000.iloc[1:,:]
    del df2000['股票代號']
    df2000 = df2000.set_index(['日期']).reset_index(drop = False)
    df2000_melt = pd.melt(df2000, id_vars= '日期', value_vars=list(df2000.columns[1:]))
    df2000_melt = df2000_melt.rename(columns={'variable':'股票代號','value':feature_names})
    df2000_melt = df2000_melt.merge(stock_name_table,on='股票代號')
    return df2000_melt


In [16]:
Volume2000 = make_melt_table("Z:\\07個人\\ChiehLin\\數據\\2000數據\\成交量2000.xlsx",'成交金額(千)')
close_price_2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\收盤價2000.xlsx','收盤價')
Lowest_price2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\最低價2000.xlsx','最低價')
Highest_price2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\最高價2000.xlsx','最高價')
open_price2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\開盤價2000.xlsx','開盤價')
Financing_used2000 = make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\融資使用率2000.xlsx','融資使用率')
Financing_retent2000 =  make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\融資維持率2000.xlsx','融資維持率')
restored_close_price2000 =  make_melt_table('Z:\\07個人\\ChiehLin\\數據\\2000數據\\還原收盤2000.xlsx','還原收盤價')

In [17]:
df_melt = Volume2000.set_index('股票名稱').reset_index(drop = False)
df_melt = df_melt.merge(close_price_2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(Lowest_price2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(Highest_price2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(open_price2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(Financing_used2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(Financing_retent2000, on= ['日期', '股票代號', '股票名稱'])
df_melt = df_melt.merge(restored_close_price2000, on= ['日期', '股票代號', '股票名稱'])

In [18]:
df_melt

,股票名稱,日期,股票代號,成交金額(千),收盤價,最低價,最高價,開盤價,融資使用率,融資維持率,還原收盤價
0,台泥,20140506,1101,279876,46.8,46.65,47.05,46.85,0.36,164.1368,24.58
1,台泥,20140507,1101,687831,46,46,46.65,46.6,0.41,162.0568,24.16
2,台泥,20140508,1101,573230,45.7,45.5,46.2,46.1,0.49,161.901,24
3,台泥,20140509,1101,599689,45.3,44.75,45.7,45.5,0.51,161.0522,23.79
4,台泥,20140512,1101,606761,44.2,44,45,44.9,0.54,158.1334,23.21
...,...,...,...,...,...,...,...,...,...,...,...
3534911,邁達康,20220622,9960,363,26,25.8,26,25.8,13.3,182.8924,26
3534912,邁達康,20220623,9960,0,25.95,25.95,25.95,25.95,13.3,182.5407,25.95
3534913,邁達康,20220624,9960,441,26,25.3,26.05,25.7,13.24,182.9065,26
3534914,邁達康,20220627,9960,338,26.35,25.7,26.35,25.8,13.24,185.3687,26.35


**<font  size=6>
建議使用:不須導出Excel表格，直接讀取Cmoney mysql數據庫數據到python內存方法介紹  
ps.節省寫出與讀取excel檔時間可以下樓抽菸
</font>**

**<font  size=5 color = "red">注意:要先在python運行環境中安裝pyodbc包，方法與安裝pandas等包相同。</font>**

以讀取20140506以後的2330台積電的為例

In [2]:
import pyodbc
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")

連結到Cmoney數據庫

In [7]:
DB_Host = "192.168.1.16"
DB_Name = "X01"
DB_User = "cm23357868"#以我的帳號為例，請填入您個人帳號
DB_Password = "86875332"#以我的密碼為例，請填入您個人密碼

print("Connecting to database using pyodbc...")

db = pyodbc.connect('DRIVER={ODBC Driver 17 for SQL Server}; SERVER=' + DB_Host + '; DATABASE=' + DB_Name + '; UID=' + DB_User + '; PWD=' + DB_Password)

print("Succesfully Connected to database using pyodbc!")

cursor = db.cursor()

Connecting to database using pyodbc...
Succesfully Connected to database using pyodbc!


找到融資使用率與融資維持率的表格路徑為[X01].[dbo].[日融資券排行]

In [3]:
query = """SELECT  [日期]
      ,[股票代號]
      ,[股票名稱]
      ,[資使用率]
      ,[融資維持率(%)]
  FROM [X01].[dbo].[日融資券排行] WITH(NOLOCK) where 股票代號 = '2330' And 日期 >= 20140506 order by 日期 """  

#FROM後填入表格路徑  WHERE指令可以選定只讀取部分數據的條件 order by 可按照特定行排序 若要使用降序 可以寫成 order by 日期 desc
#延伸閱讀可Google "Sql語法" ，也可以問我

df_Financing = pd.read_sql(query, db)

In [4]:
df_Financing

,日期,股票代號,股票名稱,資使用率,融資維持率(%)
0,20140506,2330,台積電,0.29,167.236099
1,20140507,2330,台積電,0.28,167.917099
2,20140508,2330,台積電,0.28,169.171997
3,20140509,2330,台積電,0.28,169.747406
4,20140512,2330,台積電,0.28,167.574493
...,...,...,...,...,...
2009,20220718,2330,台積電,0.45,161.492401
2010,20220719,2330,台積電,0.44,160.136002
2011,20220720,2330,台積電,0.44,161.610992
2012,20220721,2330,台積電,0.44,163.639206


**<font  size=4>可以導入多個數據集，再使用上述pd.merge方法合併</font>**

In [5]:
query = """SELECT [日期]
      ,[股票代號]
      ,[股票名稱]
      ,[開盤價]
      ,[最高價]
      ,[最低價]
      ,[收盤價]
      ,[成交金額(千)]
  FROM [X01].[dbo].[日收盤表排行] WITH(NOLOCK) where 股票代號 = '2330' And 日期 >= 20140506 order by 日期 """  

#FROM後填入表格路徑  
# WHERE指令可以選定只讀取部分數據的條件 order by 可按照特定行排序 若要使用降序 可以寫成 order by 日期 desc
#延伸閱讀可Google "Sql語法" ，也可以問我

df_price = pd.read_sql(query, db)

In [6]:
df_price

,日期,股票代號,股票名稱,開盤價,最高價,最低價,收盤價,成交金額(千)
0,20140506,2330,台積電,117.0,119.0,116.5,118.0,3247100
1,20140507,2330,台積電,118.0,119.0,117.5,118.5,3931217
2,20140508,2330,台積電,119.5,120.0,119.0,119.5,3594265
3,20140509,2330,台積電,120.0,120.0,119.0,120.0,2785145
4,20140512,2330,台積電,119.5,120.0,117.5,118.5,4390300
...,...,...,...,...,...,...,...,...
2010,20220719,2330,台積電,487.5,493.0,486.5,491.0,10063256
2011,20220720,2330,台積電,502.0,503.0,491.5,495.0,18330197
2012,20220721,2330,台積電,496.0,501.0,494.0,501.0,11887266
2013,20220722,2330,台積電,505.0,505.0,500.0,503.0,13525084


In [13]:
query = """SELECT [日期]
      ,[股票代號]
      ,[股票名稱]
      ,[還原收盤價]
  FROM [X01].[dbo].[日報酬率比較表] WITH(NOLOCK) where 股票代號 = '2330' And 日期 >= 20140506 order by 日期 """  

df_res_price = pd.read_sql(query, db)

In [8]:
df_from_sql = df_price.merge(df_Financing, on= ['日期', '股票代號', '股票名稱'])
df_from_sql = df_from_sql.merge(df_res_price, on= ['日期', '股票代號', '股票名稱'])

In [9]:
df_from_sql
#效果一樣但不用寫出與讀取Excel

,日期,股票代號,股票名稱,開盤價,最高價,最低價,收盤價,成交金額(千),資使用率,融資維持率(%),還原收盤價
0,20140506,2330,台積電,117.0,119.0,116.5,118.0,3247100,0.29,167.236099,90.64
1,20140507,2330,台積電,118.0,119.0,117.5,118.5,3931217,0.28,167.917099,91.02
2,20140508,2330,台積電,119.5,120.0,119.0,119.5,3594265,0.28,169.171997,91.79
3,20140509,2330,台積電,120.0,120.0,119.0,120.0,2785145,0.28,169.747406,92.17
4,20140512,2330,台積電,119.5,120.0,117.5,118.5,4390300,0.28,167.574493,91.02
...,...,...,...,...,...,...,...,...,...,...,...
2009,20220718,2330,台積電,495.5,498.5,488.5,495.5,20142703,0.45,161.492401,495.50
2010,20220719,2330,台積電,487.5,493.0,486.5,491.0,10063256,0.44,160.136002,491.00
2011,20220720,2330,台積電,502.0,503.0,491.5,495.0,18330197,0.44,161.610992,495.00
2012,20220721,2330,台積電,496.0,501.0,494.0,501.0,11887266,0.44,163.639206,501.00


**<font  size=4>把字符串轉換成日期</font>**

In [10]:
df_from_sql['日期'] = pd.to_datetime(df_from_sql['日期'],format = "%Y%m%d")

In [11]:
type(df_from_sql['日期'][0])

pandas._libs.tslibs.timestamps.Timestamp

In [12]:
#延伸操作，熟練pd.to_datetime 與 datetime包格式的轉換，我們可以再討論
df_from_sql['日期'][0]

Timestamp('2014-05-06 00:00:00')

In [13]:
#可以直接加減
df_from_sql['日期'][0]-df_from_sql['日期'][100]

Timedelta('-143 days +00:00:00')

In [14]:
#可以使用字符串做邏輯判斷
df_from_sql[df_from_sql['日期'] == '2014-05-06']

,日期,股票代號,股票名稱,開盤價,最高價,最低價,收盤價,成交金額(千),資使用率,融資維持率(%),還原收盤價
0,2014-05-06,2330,台積電,117.0,119.0,116.5,118.0,3247100,0.29,167.236099,90.64


**<font  size=4>將日期設為index</font>**

In [15]:
df_from_sql = df_from_sql.set_index('日期')

In [16]:
df_from_sql

,股票代號,股票名稱,開盤價,最高價,最低價,收盤價,成交金額(千),資使用率,融資維持率(%),還原收盤價
日期,,,,,,,,,,
2014-05-06,2330,台積電,117.0,119.0,116.5,118.0,3247100,0.29,167.236099,90.64
2014-05-07,2330,台積電,118.0,119.0,117.5,118.5,3931217,0.28,167.917099,91.02
2014-05-08,2330,台積電,119.5,120.0,119.0,119.5,3594265,0.28,169.171997,91.79
2014-05-09,2330,台積電,120.0,120.0,119.0,120.0,2785145,0.28,169.747406,92.17
2014-05-12,2330,台積電,119.5,120.0,117.5,118.5,4390300,0.28,167.574493,91.02
...,...,...,...,...,...,...,...,...,...,...
2022-07-18,2330,台積電,495.5,498.5,488.5,495.5,20142703,0.45,161.492401,495.50
2022-07-19,2330,台積電,487.5,493.0,486.5,491.0,10063256,0.44,160.136002,491.00
2022-07-20,2330,台積電,502.0,503.0,491.5,495.0,18330197,0.44,161.610992,495.00


**<font  size=4>生成字典,以日期為keys</font>**

In [17]:
dict_data = dict(zip(df_from_sql.index,np.array(df_from_sql)))

In [22]:
dict_data[pd.to_datetime('2014-05-06')]

array(['2330', '台積電', 117.0, 119.0, 116.5, 118.0, 3247100,
       0.28999999165534973, 167.23609924316406, 90.64], dtype=object)

In [23]:
dict_data[pd.to_datetime('2014-05-06')][0]

'2330'

In [21]:
for feature in dict_data[pd.to_datetime('2014-05-06')]:
    print(feature)

2330
台積電
117.0
119.0
116.5
118.0
3247100
0.28999999165534973
167.23609924316406
90.64


**<font  size=4>循環導出股票代號數據</font>**

In [5]:
#假設我有個列表(tuple也可以)裡面有股票代號，示範只導入前十項

s_code_df = pd.read_excel("Z:/07個人/ChiehLin/TCRINEW.xlsx")
stock_ls = list(s_code_df.loc[:10,'NUM'])

In [6]:
stock_ls

[1101, 1102, 1103, 1104, 1108, 1109, 1110, 1201, 1203, 1210, 1215]

In [5]:
from tqdm import tqdm

**<font  size=4>將不同股票的數據集存入一個列表</font>**

In [14]:
data_ls = []
for stock_code in tqdm(stock_ls):
    query = """SELECT  [日期]
      ,[股票代號]
      ,[股票名稱]
      ,[資使用率]
      ,[融資維持率(%%)]
    FROM [X01].[dbo].[日融資券排行] WITH(NOLOCK) where 股票代號 = '%s' And 日期 >= 20140506 order by 日期 """ %stock_code
    df_Financing = pd.read_sql(query, db)
    data_ls.append(df_Financing)

100%|██████████| 11/11 [00:01<00:00,  8.66it/s]


In [16]:
data_ls

[            日期  股票代號 股票名稱  資使用率    融資維持率(%)
 0     20140506  1101   台泥  0.36  164.136795
 1     20140507  1101   台泥  0.41  162.056793
 2     20140508  1101   台泥  0.49  161.901001
 3     20140509  1101   台泥  0.51  161.052200
 4     20140512  1101   台泥  0.54  158.133408
 ...        ...   ...  ...   ...         ...
 2012  20220721  1101   台泥  0.55  152.717499
 2013  20220722  1101   台泥  0.55  152.080994
 2014  20220725  1101   台泥  0.55  152.948898
 2015  20220726  1101   台泥  0.53  152.910599
 2016  20220727  1101   台泥  0.52  153.272598
 
 [2017 rows x 5 columns],
             日期  股票代號 股票名稱  資使用率    融資維持率(%)
 0     20140506  1102   亞泥  0.30  168.605194
 1     20140507  1102   亞泥  0.30  166.891495
 2     20140508  1102   亞泥  0.31  167.281097
 3     20140509  1102   亞泥  0.30  167.914703
 4     20140512  1102   亞泥  0.31  167.262497
 ...        ...   ...  ...   ...         ...
 2012  20220721  1102   亞泥  0.11  166.161697
 2013  20220722  1102   亞泥  0.11  167.455200
 2014  20220725  1102   亞泥 

**<font  size=4>也可以合併成一個數據庫</font>**

In [17]:
data_frame = pd.DataFrame()
for stock_code in tqdm(stock_ls):
    query = """SELECT  [日期]
      ,[股票代號]
      ,[股票名稱]
      ,[資使用率]
      ,[融資維持率(%%)]
    FROM [X01].[dbo].[日融資券排行] WITH(NOLOCK) where 股票代號 = '%s' And 日期 >= 20140506 order by 日期 """ %stock_code
    df_Financing = pd.read_sql(query, db)
    data_frame = pd.concat([data_frame,df_Financing],axis = 0)

100%|██████████| 11/11 [00:01<00:00,  8.29it/s]


In [18]:
data_frame

,日期,股票代號,股票名稱,資使用率,融資維持率(%)
0,20140506,1101,台泥,0.36,164.136795
1,20140507,1101,台泥,0.41,162.056793
2,20140508,1101,台泥,0.49,161.901001
3,20140509,1101,台泥,0.51,161.052200
4,20140512,1101,台泥,0.54,158.133408
...,...,...,...,...,...
2012,20220721,1215,卜蜂,0.40,156.526703
2013,20220722,1215,卜蜂,0.42,155.951599
2014,20220725,1215,卜蜂,0.40,158.622696
2015,20220726,1215,卜蜂,0.39,162.582001


**<font  size=4>將不同股票的數據集存成一個字典</font>**

In [ ]:
s_code_df = pd.read_excel("Z:/07個人/ChiehLin/TCRINEW.xlsx")
stock_ls = list(s_code_df.loc[:10,'NUM'])

In [12]:
data_ls = []
for stock_code in tqdm(stock_ls):
    query = """SELECT  [日期]
      ,[股票代號]
      ,[股票名稱]
      ,[資使用率]
      ,[融資維持率(%%)]
    FROM [X01].[dbo].[日融資券排行] WITH(NOLOCK) where 股票代號 = '%s' And 日期 >= 20140506 order by 日期 """ %stock_code
    df_Financing = pd.read_sql(query, db)
    data_ls.append(df_Financing)   

100%|██████████| 11/11 [00:01<00:00,  7.77it/s]


In [9]:
data_dict = {str(stock_ls[i]):data_ls[i] for i in range(len(stock_ls))}

**<font  size=4>將不同股票的數據集與樞紐數據集存成兩個不同的字典</font>**

In [44]:
data_ls = []
data_frame = pd.DataFrame()
for stock_code in tqdm(stock_ls):
    query = """SELECT  [日期]
      ,[股票代號]
      ,[股票名稱]
      ,[資使用率]
      ,[融資維持率(%%)]
    FROM [X01].[dbo].[日融資券排行] WITH(NOLOCK) where 股票代號 = '%s' And 日期 >= 20140506 order by 日期 """ %stock_code
    df_Financing = pd.read_sql(query, db)
    data_ls.append(df_Financing)
    data_frame = pd.concat([data_frame,df_Financing],axis = 0)
    
feature_ls = data_frame.columns[3:]
data_pivots_ls = []
for feature in feature_ls:
    data_pivot = data_frame.pivot('日期','股票代號',feature)
    data_pivots_ls.append(data_pivot)

100%|██████████| 11/11 [00:01<00:00,  8.05it/s]


In [45]:
#股票數據集字典
data_dict = {str(stock_ls[i]):data_ls[i] for i in range(len(stock_ls))}

In [46]:
data_dict['1101']

,日期,股票代號,股票名稱,資使用率,融資維持率(%)
0,20140506,1101,台泥,0.36,164.136795
1,20140507,1101,台泥,0.41,162.056793
2,20140508,1101,台泥,0.49,161.901001
3,20140509,1101,台泥,0.51,161.052200
4,20140512,1101,台泥,0.54,158.133408
...,...,...,...,...,...
2020,20220802,1101,台泥,0.52,160.040604
2021,20220803,1101,台泥,0.55,160.391403
2022,20220804,1101,台泥,0.53,159.009598
2023,20220805,1101,台泥,0.53,159.522598


In [47]:
#樞紐數據集字典
pivot_dict = {str(feature_ls[i]):data_pivots_ls[i] for i in range(len(feature_ls))}

In [48]:
pivot_dict['融資維持率(%)']

股票代號,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,1215
日期,,,,,,,,,,,
20140506,164.136795,168.605194,167.481293,159.481201,157.666794,190.517700,163.123703,153.523895,153.771301,165.095993,176.331696
20140507,162.056793,166.891495,166.957504,158.392700,157.498795,189.149704,160.656601,154.247192,155.477493,164.053894,170.040802
20140508,161.901001,167.281097,166.436096,160.124405,157.874405,187.212997,161.677094,154.150406,155.149994,165.724594,166.997406
20140509,161.052200,167.914703,165.410904,158.235901,158.790604,187.186401,160.246094,153.466202,154.555206,165.733704,166.979202
20140512,158.133408,167.262497,165.413803,156.172501,158.236694,187.831894,161.032196,150.103104,154.278793,164.447906,173.233002
...,...,...,...,...,...,...,...,...,...,...,...
20220802,160.040604,156.431503,145.347900,165.633102,139.130203,156.607498,151.208496,152.507202,166.620697,155.057907,162.679092
20220803,160.391403,156.640701,144.913300,163.755798,136.541702,156.688705,151.629700,153.348694,164.900803,153.850906,161.442001
20220804,159.009598,154.242996,140.262405,163.389404,138.003799,143.370300,150.791306,151.923798,164.652893,154.195404,159.792496


**<font  size=4>當然，也可以將兩個字典合併，讓兩種呼叫方式都可以在同一個字典使用</font>**

In [49]:
data_dict.update(pivot_dict)

In [52]:
data_dict['1101']

,日期,股票代號,股票名稱,資使用率,融資維持率(%)
0,20140506,1101,台泥,0.36,164.136795
1,20140507,1101,台泥,0.41,162.056793
2,20140508,1101,台泥,0.49,161.901001
3,20140509,1101,台泥,0.51,161.052200
4,20140512,1101,台泥,0.54,158.133408
...,...,...,...,...,...
2020,20220802,1101,台泥,0.52,160.040604
2021,20220803,1101,台泥,0.55,160.391403
2022,20220804,1101,台泥,0.53,159.009598
2023,20220805,1101,台泥,0.53,159.522598


In [53]:
data_dict['融資維持率(%)']

股票代號,1101,1102,1103,1104,1108,1109,1110,1201,1203,1210,1215
日期,,,,,,,,,,,
20140506,164.136795,168.605194,167.481293,159.481201,157.666794,190.517700,163.123703,153.523895,153.771301,165.095993,176.331696
20140507,162.056793,166.891495,166.957504,158.392700,157.498795,189.149704,160.656601,154.247192,155.477493,164.053894,170.040802
20140508,161.901001,167.281097,166.436096,160.124405,157.874405,187.212997,161.677094,154.150406,155.149994,165.724594,166.997406
20140509,161.052200,167.914703,165.410904,158.235901,158.790604,187.186401,160.246094,153.466202,154.555206,165.733704,166.979202
20140512,158.133408,167.262497,165.413803,156.172501,158.236694,187.831894,161.032196,150.103104,154.278793,164.447906,173.233002
...,...,...,...,...,...,...,...,...,...,...,...
20220802,160.040604,156.431503,145.347900,165.633102,139.130203,156.607498,151.208496,152.507202,166.620697,155.057907,162.679092
20220803,160.391403,156.640701,144.913300,163.755798,136.541702,156.688705,151.629700,153.348694,164.900803,153.850906,161.442001
20220804,159.009598,154.242996,140.262405,163.389404,138.003799,143.370300,150.791306,151.923798,164.652893,154.195404,159.792496
